### Contents:
* Preparing the ground
    * Importing Libs and Datasets
    * Data check and preprocessing
* Feature engineering
* Baseline model
* More models

### Importing libs and datasets

In [4]:
pip install pipreqs

Note: you may need to restart the kernel to use updated packages.
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=4f6269b816699a56549743141c63a8ecdce113bec13ef2b874c5da9555506d33
  Stored in directory: c:\users\morea\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [34]:
import pandas as pd
import numpy as np
import os
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS
from IPython.display import display, HTML
from sklearn.preprocessing import MinMaxScaler
import scipy
from scipy.sparse import hstack
from sklearn.svm import LinearSVC#,SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,roc_auc_score,confusion_matrix
import pickle
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv("D:/Programming/DB's/Toxic_database/tox_train.csv")
#10 sec loading

### Data check and preprocessing

# heatmap target vs features??

In [3]:
df.drop_duplicates(keep=False,subset=['comment_text'],inplace=True)#Dataset duplicates are removed
print(df.comment_text.tail(1))
df.reset_index(drop=True,inplace=True)#Dropping empty ID's by resetting indexation. Now the last ID is the same as the number of comments.
print(df.comment_text.tail(1))

1804873    Students defined as EBD are legally just as di...
Name: comment_text, dtype: object
1770642    Students defined as EBD are legally just as di...
Name: comment_text, dtype: object


* Dataset duplicates are removed.
* Dropping empty ID's by resetting indexation. Now the last ID is the same as the number of comments.

In [4]:
df['target_class']=(df['target']>=0.5).map(int)#if more than .5 - than toxic.
X_train, X_rem, y_train, y_rem = train_test_split(df['comment_text'], df['target_class'],
                                                    stratify= df['target_class'], 
                                                    test_size=0.20)
X_test,X_valid,y_test,y_valid=train_test_split(X_rem, y_rem,
                                                    stratify= y_rem, 
                                                    test_size=0.50)

* Creating Rough toxic classification based on 0.5 target threshold to count clean and toxic comments (class imbalance).
* Split to train/test/validation (X,y, by='y')

### Feature engineering

In [6]:
def adding_inderect_features(df):
    df=df.to_frame(name='comment_text')
    df.loc[:,'count_word']=df["comment_text"].apply(lambda x: len(str(x).split()))
    df.loc[:,'count_unique_word']=df["comment_text"].apply(lambda x: len(set(str(x).split())))
    df.loc[:,'count_letters'] = df["comment_text"].apply(lambda x: len(str(x)))
    df.loc[:,"count_punctuations"] = df["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    df.loc[:,"count_words_upper"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    df.loc[:,"count_words_title"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
    df.loc[:,"count_stopwords"] = df["comment_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
    df.loc[:,"mean_word_len"] = df["comment_text"].apply(lambda x: round(np.mean([len(w) for w in str(x).split()]),2))
    df.loc[:,'word_unique_percent']=df.loc[:,'count_unique_word']*100/df['count_word']
    df.loc[:,'punct_percent']=df.loc[:,'count_punctuations']*100/df['count_word']
    return df

In [7]:
tfidf=TfidfVectorizer(ngram_range=(1,1),max_df=0.8,min_df=10)
tfidf_X_train=tfidf.fit_transform(X_train)
tfidf_X_test=tfidf.transform(X_test)
X_train_w_inderect_f=adding_inderect_features(X_train)
X_test_w_inderect_f = adding_inderect_features(X_test)
inderect_f=set(list(X_train_w_inderect_f.columns))-set(list(X_train.to_frame(name='comment_text')))
scaler_1=MinMaxScaler()# also add Scale, norm
scaled_X_train_only_features = scaler_1.fit_transform(X_train_w_inderect_f[inderect_f])# numpy.ndarray 10 normalized features 
scaled_X_test_only_features = scaler_1.fit_transform(X_test_w_inderect_f[inderect_f])
sparce_scaled_X_train_only_features=scipy.sparse.csr_matrix(scaled_X_train_only_features)#convert np matrix to csr matrix 
# 1416514x10 numpy.ndarray to sparse matrix numpy.float64
sparce_scaled_X_test_only_features=scipy.sparse.csr_matrix(scaled_X_test_only_features)#convert np matrix to csr matrix 
sparce_train=hstack((tfidf_X_train, sparce_scaled_X_train_only_features))
sparce_test=hstack((tfidf_X_test, sparce_scaled_X_test_only_features))

In [8]:
print(f'Train TFIDF size: {tfidf_X_train.shape, type(tfidf_X_train)}')
print(f'Train Indirect features size: {sparce_scaled_X_train_only_features.shape, type(sparce_scaled_X_train_only_features)}')
print(f'Train result of TFIDF+Indirect_features: {sparce_train.shape, type(sparce_train)}')
print(f'\n\nTest TFIDF size: {tfidf_X_test.shape, type(tfidf_X_test)}')
print(f'Test Indirect features size: {sparce_scaled_X_test_only_features.shape, type(sparce_scaled_X_test_only_features)}')
print(f'Test result of TFIDF+Indirect_features: {sparce_test.shape, type(sparce_test)}')

Train TFIDF size: ((1416514, 57263), <class 'scipy.sparse.csr.csr_matrix'>)
Train Indirect features size: ((1416514, 10), <class 'scipy.sparse.csr.csr_matrix'>)
Train result of TFIDF+Indirect_features: ((1416514, 57273), <class 'scipy.sparse.coo.coo_matrix'>)


Test TFIDF size: ((177064, 57263), <class 'scipy.sparse.csr.csr_matrix'>)
Test Indirect features size: ((177064, 10), <class 'scipy.sparse.csr.csr_matrix'>)
Test result of TFIDF+Indirect_features: ((177064, 57273), <class 'scipy.sparse.coo.coo_matrix'>)


In [3]:
def metrics(y_true,y_pred):
    y_true = y_test
    target_names = ['Toxic', 'Not_toxic']
    print(classification_report(y_true, y_pred, target_names=target_names))
    
    roc_auc_score(y_true=y_true,y_score=y_pred,average='samples')
    print(f"Area Under the Curve score: {round(roc_auc_score(y_true,y_pred),2)} \
    \n Is this probability of Toxic or not toxic? \
    \n or it's a probability of 1 (Toxic) class to be toxic?")
    
    print('\n',confusion_matrix(y_true=y_true, y_pred=y_pred))
    print('\n',np.array([['true negatives','false negatives'],['true positives','false positives']] ))
    print('\n',np.array([['correct non-toxic predict','wrong not-toxic prediction'],['correct toxic prediction','wrong toxic prediction']] ))
    
# y_pred=model.predict(sparce_test)
# metrics(y_test,y_pred)

In [5]:
#save_splitted_df's
import scipy.sparse
os.makedirs('D:/Programming/Repositories/toxic_detection_classification/Model', exist_ok=True)
X_train.to_csv('D:/Programming/Repositories/toxic_detection_classification/Model/X_train.csv',index=False)
y_train.to_csv('D:/Programming/Repositories/toxic_detection_classification/Model/y_train.csv',index=False)
X_test.to_csv('D:/Programming/Repositories/toxic_detection_classification/Model/X_test.csv',index=False)
y_test.to_csv('D:/Programming/Repositories/toxic_detection_classification/Model/y_test.csv',index=False)

scipy.sparse.save_npz("sparce_train.npz", sparce_train)#save sparce matrix 
scipy.sparse.save_npz("sparce_test.npz", sparce_test)

In [2]:
# load from disc
X_train = pd.read_csv("X_train.csv").iloc[:,0]
y_train = pd.read_csv("y_train.csv").iloc[:,0]
X_test = pd.read_csv("X_test.csv").iloc[:,0]
y_test = pd.read_csv("y_test.csv").iloc[:,0]
sparce_train = scipy.sparse.load_npz("sparce_train.npz")
sparce_test = scipy.sparse.load_npz("sparce_test.npz")

### Baseline model

In [ ]:
df_result = pd.DataFrame(columns=["model", "description", "dataset_type"])

In [8]:
def save_model(model,filename='model.sav'):
    """save the model to disk"""
    pickle.dump(model, open(filename, 'wb'))

In [21]:
#fit and save
linearSVC = LinearSVC(random_state=0, tol=1e-5)
LinSVC_fitted=linearSVC.fit(sparce_train, y_train)#(1416514, 57204)
save_model(LinSVC_fitted,'LinearSVC_model.sav')

In [4]:
# load the model from disk
linearSVC = pickle.load(open('LinearSVC_model.sav', 'rb'))
y_pred=linearSVC.predict(sparce_test)
metrics(y_test,y_pred)

              precision    recall  f1-score   support

       Toxic       0.95      0.99      0.97    162921
   Not_toxic       0.80      0.45      0.58     14143

    accuracy                           0.95    177064
   macro avg       0.88      0.72      0.78    177064
weighted avg       0.94      0.95      0.94    177064

Area Under the Curve score: 0.72     
 Is this probability of Toxic or not toxic?     
 or it's a probability of 1 (Toxic) class to be toxic?

 [[161302   1619]
 [  7710   6433]]

 [['true negatives' 'false negatives']
 ['true positives' 'false positives']]

 [['correct non-toxic predict' 'wrong not-toxic prediction']
 ['correct toxic prediction' 'wrong toxic prediction']]


In [7]:
linearSVC.predict(sparce_test)

NameError: name 'linearSVC' is not defined

* skip SVC

In [5]:
#fit and save
SVC=SVC(random_state=0,gamma='auto',kernel='linear')# processed for 10 h and freezes
# SVC_fitted=SVC.fit(sparce_train, y_train)#(1416514, 57204)

NameError: name 'SVC' is not defined

* DTC Done

In [6]:
#fit and save
DTC = DecisionTreeClassifier(random_state=0)
DTC_fitted=DTC.fit(sparce_train, y_train)
save_model(DTC_fitted,'DTC_model.sav')

NameError: name 'save_model' is not defined

In [13]:
# save the model to disk
filename = 'DTC_model.sav'
pickle.dump(DTC_fitted, open(filename, 'wb'))

In [8]:
# load the model from disk
DTC = pickle.load(open('DTC_model.sav', 'rb'))
y_pred=DTC.predict(sparce_test)
metrics(y_test,y_pred)

              precision    recall  f1-score   support

       Toxic       0.95      0.96      0.96    162921
   Not_toxic       0.49      0.45      0.47     14143

    accuracy                           0.92    177064
   macro avg       0.72      0.71      0.71    177064
weighted avg       0.92      0.92      0.92    177064

Area Under the Curve score: 0.71     
 Is this probability of Toxic or not toxic?     
 or it's a probability of 1 (Toxic) class to be toxic?

 [[156360   6561]
 [  7764   6379]]

 [['true negatives' 'false negatives']
 ['true positives' 'false positives']]

 [['correct non-toxic predict' 'wrong not-toxic prediction']
 ['correct toxic prediction' 'wrong toxic prediction']]


In [19]:
set(DTC.predict_proba(sparce_test)[:,0])

{0.0, 0.5, 0.6666666666666666, 0.9, 1.0}

In [20]:
# random forest, xgboost, lgbm

In [ ]:
import winsound
duration = 10000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

### CSV Export

In [ ]:
os.makedirs('D:/Programming/Repositories/toxic_detection_classification/Model', exist_ok=True)
df_temp.to_csv('D:/Programming/Repositories/toxic_detection_classification/Model/tox_train_featurefull')